In [1]:
import re
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import requests as rq


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MHDAH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MHDAH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#Class crawler to retrive documents from web pages
class crawler:

    def __init__(self): 
        self.to_visit = list()
        self.visited = set()

    def fetch(self, url):#to get all the content of url (return content)
        print('now fetching.. ', url)
        res = requests.get(url)

        return res.content

    def get_current_url(self):#to get all url visited (reurn url visited and print it)
        res = self.to_visit.pop(0)

        while res in self.visited:
            print('already visited', res)
            res = self.to_visit.pop(0)

        return res

    def get_links(self, current_url, content):#to get all links with http from html page
        urls = re.findall('<a href="([^"]+)">', str(content)) 
        print('urls are', urls)
        for url in urls:
            if url[0] == '/':
                url = current_url + url[1:]
            pattern = re.compile('https?')
            if pattern.match(url):
                self.to_visit.append(url) # عم ضيفها عل  to visit

    def crawl(self, url, depth=20):
        self.to_visit.append(url)
        while len(self.visited) <= depth:
            for id in range(0, len(self.visited)):
                current_url = self.get_current_url()                                                # get url from web page
                content = self.fetch(current_url)                                                   # retrive content from web page
                p_content = re.findall('<p>(.*?)</p>', str(content))                                # find paragraph of page
                data = pd.DataFrame({'Doc_ID': id, 'URL': current_url, 'Content': p_content})       # make it as dataFrame
                dataset = data.to_csv('dataset.csv', index=False)                                   # save final dataset file
                self.visited.add(current_url)
                self.get_links(current_url,content)
        return dataset

In [ ]:
#------------------------------------Testing URLs-----------------------------------
c = crawler()
c.crawl('https://noobnotes.net/')

In [4]:
def retrieve_docs_and_clean():
  get_url = rq.get("https://noobnotes.net/part-of-your-world-little-mermaid-disney/?solfege=false")
  get_text = get_url.text

  soup = BeautifulSoup(get_text, "html.parser")

  song_title = [i.text for i in soup.findAll('h1', {
    "class": "entry-title"
  })]


  div = soup.find('div', {'class': 'post-content'})
  par = div.findChildren('p', recursive=False)

  # Retrieve Paragraphs
  documents = []
  documents_clean = []
  for p in par:
       # print(p)
        documents = str(p)
        line_words = documents.partition('<br')[2]
        line_words = line_words.replace('</br>', '')
        line_words = line_words.replace('</p>', '')
        line_words = line_words.replace('/>', '')
        line_words = line_words.replace('>', '')
        line_words = line_words.replace('.', '')
        line_words = line_words.replace(',', '')
        line_words = line_words.replace('"', '')
        line_words = line_words.replace('?', '')
        line_words = line_words.replace('!', '')
        line_words = line_words.replace('~', '')
        line_words = line_words.replace('(<', '')
        line_words = line_words.replace(')', '')
        line_words = line_words.replace('? ', '')
        line_words = line_words.replace('</em', '')
        line_words = line_words.replace('-', '')
        documents_clean.append(line_words)
        
  return documents_clean
  
  

In [5]:
docs = retrieve_docs_and_clean()
print(docs)

['', "\nLook at this stuff isn't it neat", "\nWouldn't you think my collection's complete", "\nWouldn't you think I'm the girl", '\nThe girl who has everything', '', '\nLook at this trove treasures untold', '\nHow many wonders can one cavern hold', '\nLooking around here you think', "\nSure she's got everything", '', "\nI've got gadgets and gizmos aplenty", "\nI've got whozits and whatzits galore", '\nWant a thingamabob', "\nI've got twenty", '\nBut who cares No big deal', '\nI want more', '', '\nI wanna be where the people are', "\nI wanna see wanna see them dancing'", '\nWalking around on those', "\nWhat do you call 'em", '\nOh  feet', '', "\nFlippin' your fins you don't get too far", '\nLegs are required for jumping dancing', '\nStrolling along down a', "\nWhat's that word again", '\nStreet', '', '\nUp where they walk up where they run', '\nUp where they stay all day in the sun', "\nWanderin' free  wish I could be", '\nPart of that world', '', '\nWhat would I give', '\nIf I could li

In [6]:
stopWords = set(stopwords.words('english'))
cleanedData = []
for sentence in docs:
  filtered_sentence = []
  words = word_tokenize(sentence)
      
  for word in words:
      if word not in stopWords:
          filtered_sentence.append(word)
  cleanedData.append(' '.join(filtered_sentence))

In [7]:
cleanedData

['',
 "Look stuff n't neat",
 "Would n't think collection 's complete",
 "Would n't think I 'm girl",
 'The girl everything',
 '',
 'Look trove treasures untold',
 'How many wonders one cavern hold',
 'Looking around think',
 "Sure 's got everything",
 '',
 "I 've got gadgets gizmos aplenty",
 "I 've got whozits whatzits galore",
 'Want thingamabob',
 "I 've got twenty",
 'But cares No big deal',
 'I want',
 '',
 'I wan na people',
 "I wan na see wan na see dancing '",
 'Walking around',
 "What call 'em",
 'Oh feet',
 '',
 "Flippin ' fins n't get far",
 'Legs required jumping dancing',
 'Strolling along',
 "What 's word",
 'Street',
 '',
 'Up walk run',
 'Up stay day sun',
 "Wanderin ' free wish I could",
 'Part world',
 '',
 'What would I give',
 'If I could live waters',
 'What would I pay',
 'To spend day warm sand',
 '',
 "Bet'cha land understand",
 "Bet n't reprimand daughters",
 'Bright young women sick swimming',
 'Ready stand',
 '',
 "I 'm ready know people know",
 "Ask 'em que

In [8]:
def extract_unique_word(corpus):
  uniqueWords = set()
  for sentence in corpus:
    sentence = word_tokenize(sentence)
    for word in sentence:
      uniqueWords.add(word)
  return uniqueWords

In [9]:
docss=extract_unique_word(cleanedData)
print(docss)

{'Would', 'could', 'run', 'stand', 'jumping', 'sand', 'thingamabob', 'explore', 'galore', 'walk', 'many', 'Wanderin', 'feet', 'To', 'pay', 'But', 'wonders', 'stuff', 'collection', 'women', 'free', 'Up', 'girl', 'daughters', 'Street', 'twenty', 'want', 'call', 'one', 'love', 'gadgets', 'wan', 'Love', 'stay', 'around', 'world', 'Oh', 'get', "Bet'cha", 'Ready', 'think', 'trove', 'neat', 'deal', 'Sure', "n't", 'Want', 'aplenty', 'whozits', 'cares', 'people', 'fire', 'sea', 'Part', 'Walking', 'untold', 'far', 'warm', 'No', 'big', 'day', 'Look', 'What', 'Ask', 'Strolling', 'give', 'I', 'Flippin', 'required', "'em", 'Burn', 'would', 'fins', 'whatzits', 'swimming', 'land', 'dancing', 'How', "'ve", "'", 'see', 'wish', 'turn', 'na', 'complete', 'hold', 'ready', 'cavern', 'If', 'spend', 'shore', 'young', 'waters', 'know', "'m", "'s", 'understand', 'When', 'along', 'questions', 'sun', 'word', 'sick', 'answers', 'gizmos', 'treasures', 'Legs', 'Looking', 'live', 'Bright', 'Out', 'everything', 'Bet',

In [10]:
# Instantiate a TfidfVectorizer object
vectorizer = TfidfVectorizer()
# It fits the data and transform it as a vector
X = vectorizer.fit_transform(docs)
# Convert the X as transposed matrix
X = X.T.toarray()
# Create a DataFrame and set the vocabulary as the index
df = pd.DataFrame(X, index=vectorizer.get_feature_names())
df

c:\Users\MHDAH\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
above,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.41645,0.0,0.0
again,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
all,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
along,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
and,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.298403,0.360041,0.0,0.0,0.000000,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
would,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
wouldn,0.0,0.0,0.388439,0.468496,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.377852,0.00000,0.0,0.0
you,0.0,0.0,0.344944,0.416036,0.0,0.0,0.0,0.0,0.368962,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
young,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0


In [11]:
def build_index(corpus):
  index = dict()
  uniqueWords = extract_unique_word(corpus)
  for word in uniqueWords:
    temp = []
    for i, sentence in enumerate(corpus):
      sentence = word_tokenize(sentence)
      if word in sentence:
        temp.append(i)
    index[word] = temp
  return index

In [12]:
index = build_index(cleanedData)
print(index)

{'Would': [2, 3, 50], 'could': [32, 36, 52], 'run': [30], 'stand': [43], 'jumping': [25], 'sand': [38], 'thingamabob': [13], 'explore': [51], 'galore': [12], 'walk': [30], 'many': [7], 'Wanderin': [32], 'feet': [22], 'To': [38], 'pay': [37], 'But': [15], 'wonders': [7], 'stuff': [1], 'collection': [2], 'women': [42], 'free': [32], 'Up': [30, 31], 'girl': [3, 4], 'daughters': [41], 'Street': [28], 'twenty': [14], 'want': [16], 'call': [21], 'one': [7], 'love': [50], 'gadgets': [11], 'wan': [18, 19], 'Love': [51], 'stay': [31], 'around': [8, 20], 'world': [33, 53], 'Oh': [22], 'get': [24, 46], "Bet'cha": [40], 'Ready': [43], 'think': [2, 3, 8], 'trove': [6], 'neat': [1], 'deal': [15], 'Sure': [9], "n't": [1, 2, 3, 24, 41, 50], 'Want': [13], 'aplenty': [11], 'whozits': [12], 'cares': [15], 'people': [18, 45], 'fire': [47], 'sea': [52], 'Part': [33, 53], 'Walking': [20], 'untold': [6], 'far': [24], 'warm': [38], 'No': [15], 'big': [15], 'day': [31, 38], 'Look': [1, 6], 'What': [21, 27, 35,

In [13]:
def and_serach(query):
  queryWords = word_tokenize(query)
  result = []
  for queryWord in queryWords:
    if queryWord in list(index.keys()):
      result.append(index[queryWord])
  i = 0 
  finalResult = []
  for j in range(1,len(result)):
    finalResult.append(set(result[i]) & set(result[j]))
    i = j
  return finalResult

In [16]:
and_serach("call What")

[]

In [ ]:
def or_serach(query):
  queryWords = word_tokenize(query)
  result = []
  for queryWord in queryWords:
    if queryWord in list(index.keys()):
      result.append(index[queryWord])
  finalResult = set()
  for document in result:
    for i in document:
      finalResult.add(i)
  return finalResult

In [ ]:
or_serach("call What")

In [ ]:
def dynamic_query(query):
  orginal_index = list(range(0,len(index)))
  query = query.split()
  result = []
  expression = ''
  for i,word in enumerate(query):
    if i == 1:
      expression = query[i-1] + " " + query[i+1]
      #print(expression)
      if word == 'and':
        result = and_serach(expression)
        #print(result)
      elif word == 'or':
        result = or_serach(expression)
    elif i > 1:
      if word == 'and':
        result = result & index[query[i+1]]
      elif word == 'or' :
        result = result + index[query[i+1]]
      elif word == 'not':
        indexes = index[query[i+1]]
        not_ = list(set(orginal_index) - set(indexes))
        result = result[-1] & set(not_)
  return result

In [ ]:
dynamic_query("call and What or untold")

In [ ]:
from nltk.corpus import wordnet as wn
wn.synsets('call')#a set of synonyms that share a common meaning

In [ ]:
dog = wn.synset('dog.n.01')
cat = wn.synset('cat.n.01')

wn.path_similarity(dog, cat)# Return a score denoting how similar two word senses are, based on the shortest path that connects the senses in the is-a (hypernym/hypnoym) taxonomy